
![ImageName](pic/title.jpg)  

**作者：[lqy](https://www.heywhale.com/home/user/profile/5f27fd9633e1be002cc65a1d)** 华东师范大学气象学研究生、和鲸社区气象数据科学频道版主

🐋：本项目来自和鲸社区《[气象训练营⑦：WRF模式后处理](https://www.heywhale.com/home/competition/64478fec113e81a18dc70cd1)》活动，所有教案代码都可以一键跑通，你可以 fork 后在线在线运行、调试学习、完成作业练习。  

学习过程中如果你遇到任何问题，欢迎使用搜索引擎，或在 [讨论区](https://www.heywhale.com/home/competition/forumlist/64478fec113e81a18dc70cd1) 中发帖提出，我们很乐意为你提供帮助。

# 关卡五：WRF模拟数据的插值  


数据插值是一种空间数据的常用处理方式，其**不仅可以是时间序列插值，也可以是空间分布插值**；从插值的输入和输出数据可以分为，**点数据到格点数据、格点数据到格点数据（重采样）、格点数据到点数据（数据提取）** 三种形式；从插值的方法，可以分为**线性插值、反距离权重差值、克里金插值**等等。  

这一关相对难度较大、内容较多，但提供了很多素材和示例，能够满足多种的插值场景需要。

## 一、关于空间插值的基本概念

### 1. 什么是空间插值  

**插值（Interpolation）** 是一个从已知点近似计算未知点的近似计算方法。    
    
**空间插值（Spatial Interpolation）** 是将离散点的测量数据转换为连续的数据曲面，以便与其它空间现象的分布模式进行比较，它包括了**空间内插和外推**两种算法。    
    
**空间重插值，又可以被称为空间重采样，** 实现数据在不同空间分辨率下的转换。    
    


### 2. 空间插值的常用方法  


**线性插值**：将模式层的数据点在垂直方向上进行线性插值，然后再按照地面高度和气压的关系，将插值结果映射到地面层。这是一种简单且广泛使用的方法，但可能会引入较大误差，特别是在高地形和不均匀地表覆盖的地区。    
**双线性插值**：对于水平均匀网格的气象模式输出数据，可以使用双线性插值将模式层数据插值到地面层。这种方法可以保留更多的信息和精度，但需要确保网格分辨率足够高。    
    
需要注意的是，**不同插值方法和参数设置可能会导致不同的插值误差和结果**。通常 WRF 模拟数据并不存在缺失值，线性插值方法能最大程度保留原始模拟特征。

### 3. 关于WRF模拟数据的插值  

对于WRF数据的插值，本质上也是空间插值。但是不同于传统的空间数据，WRF模拟数据的维度更高（经度、纬度、时间、高度），插值形式也更为丰富，主要可以分为以下四种类型：  
（1）**格点到格点插值（重插值）**：非规则网格转换为规则网格  
（2）**格点到站点插值**：根据站点提取格点的物理量  
（3）**水平层面的插值**：获取特定气压层面的物理量  
（4）**垂直层面的插值**：获取物理量的垂直分布特征  


In [ ]:
# 导入模块
from scipy import interpolate
import metpy.calc as mpcalc
from metpy.units import units
import xesmf as xe
import xarray as xr
import salem
import numpy as np
import matplotlib.path as mpath
import matplotlib.pyplot as plt
import matplotlib as mpl
from wrf import getvar, interplevel, CoordPair, vertcross, ll_to_xy, xy_to_ll
from netCDF4 import Dataset

In [ ]:
# WRF数据目录
wrfout_path = './dataset/wrfout/'

In [ ]:
# 读取 WRF 模拟数据
wrf_file = Dataset(wrfout_path + 'wrfout_d01_2019-08-09_06_00_00')

## 二、WRF模拟数据的插值场景

对于 WRF 模拟数据而言，我们通常有以下五种插值场景：  
**1、插值到指定的高度层  
2、从地面到高空的垂直插值  
3、三维差值  
4、二维到一维插值  
5、非规则网格到规则网格插值**

### 1. 插值到指定的高度层

wrf-python中`interplevel`函数可以将三维数据插值到水平层上，通常是指定的高度层。

In [ ]:
# 提取位势高度
z = getvar(wrf_file, 'z')
# 提取高度层
p = getvar(wrf_file, 'pressure')
# 计算500 hPa位势高度
ht_500 = interplevel(z, p, 500)

### 2. 从地面到高空的垂直插值

将模式层的数据按照**地面高度插值到一系列垂直层次**，然后再将这些垂直层次的数据插值到地面层。这种方法可以考虑到大气的垂直结构和稳定性，但需要考虑到下垫面的高度和物理特征。    
    
wrf-python中`vertcross`函数可以用来创建垂直剖面图。**为了定义垂直剖面，需要指定剖面的起始和终止点，也可以提供中心点和角度来进行剖面。**    
    
可以使用wrf-python中`CoordPair`对象指定起始，终止或中心点。坐标点也可以是 (x, y) 网格点或是经纬度坐标点。    
    
当使用经纬度坐标时，需要提供 `netCDF`文件对象或是`wrf.WrfProj`对象。    
    
**垂直层可以通过 levels 参数指定**。

#### 使用网格点坐标

In [ ]:
# 根于网格坐标定义起点和终点坐标
start_point = CoordPair(x=0, y=(z.shape[-2]-1)//2)
end_point = CoordPair(x=-1, y=(z.shape[-2]-1)//2)

# 计算垂直剖面
# 通过设置 latlon = True，将沿着剖面线计算经纬度坐标 
# 并且添加经纬度坐标到 xy_loc 元数据，从而帮助绘图
p_vert = vertcross(p, z, start_point=start_point, end_point=end_point, latlon=True)
p_vert

#### 使用中心点和角度

In [ ]:
# 在网格坐标中定义中心点和角度, 中心点在网格的中心
pivot_point = CoordPair(x=(z.shape[-1]-1)//2, y=(z.shape[-2]-1)//2)
angle = 90.0

p_vert = vertcross(p, z, pivot_point=pivot_point, angle=angle, latlon=True)
p_vert

#### 使用经纬度坐标

In [ ]:
# 获取 WRF 模拟的经纬度
lats = getvar(wrf_file, 'lat')
lons = getvar(wrf_file, 'lon')

# 使用相同的水平线，但是分别为纬度和经度
start_lat = lats[(lats.shape[-2]-1)//2, 0]
end_lat = lats[(lats.shape[-2]-1)//2, -1]
start_lon = lons[(lats.shape[-2]-1)//2, 0]
end_lon = lons[(lats.shape[-2]-1)//2, -1]

# 创建剖面线
start_point = CoordPair(lat=start_lat, lon=start_lon)
end_point = CoordPair(lat=end_lat, lon=end_lon)

# 使用经纬度坐标时，必须提供netcdf文件对象或投影对象
p_vert = vertcross(p, z, wrfin=wrf_file, start_point=start_point, end_point=end_point, latlon=True)
p_vert

#### 指定垂直层

In [ ]:
# 指定垂直层（单位为m）
levels = [500, 1000, 1500, 2000, 3000]

# 获取不同高度的气压值
p_vert = vertcross(p, z, wrfin=wrf_file, levels=levels, start_point=start_point, end_point=end_point, latlon=True)
p_vert

**下垫面插值是将气象模式数据插值到地面层的过程**，本质上也是垂直插值的一种。因为气象模式数据通常是以**垂直层次（pressure levels）** 输出，而地面气象观测数据是在地面测量得到的，两者之间存在高度差和温度、湿度等物理量的变化。

### 3. 三维插值

利用wrf-python中的`vinterp`函数可以对**经度、纬度和高度**三个维度同时进行插值。

In [ ]:
from wrf import getvar, vinterp

tk = getvar(wrf_file, "tk")
# 插值 tk 到 theta-e 层
interp_levels = [200, 300, 500, 1000]

interp_field = vinterp(wrf_file,
                       field=tk,
                       vert_coord="eth",
                       interp_levels=interp_levels,
                       extrapolate=True,
                       field_type="tk",
                       log_p=True)
interp_field

### 4. 二维到一维插值

使用`wrf.interpline`函数可以沿着一条线对2D场进行插值，这类似3D场的垂直剖面插值。为了定义插值的线，可以是线的起始和终止点。当然，也可以提供中心点和角度来进行剖面。可以使用`wrf.CoordPair`对象指定起始，终止或中心点。坐标点也可以是 (x, y) 网格点或是经纬度坐标点。当使用经纬度坐标时，需要提供netCDF文件对象或是`wrf.WrfProj`对象。

#### 使用网格点坐标

In [ ]:
from wrf import interpline

# 获取 2m 温度
t2 = getvar(wrf_file, 'T2')

# 使用起始和终止点在区域中心创建南北线
start_point = CoordPair(x=(t2.shape[-1]-1)//2, y=0)
end_point = CoordPair(x=(t2.shape[-1]-1)//2, y=-1)

# 通过设置 latlon = True，将沿着线计算经纬度坐标 
# 并且添加经纬度坐标到 xy_loc 元数据，从而帮助绘图
t2_line = interpline(t2, start_point=start_point, end_point=end_point, latlon=True)
t2_line

#### 使用中心点和角度

In [ ]:
# 使用中心点和角度创建南北线
pivot_point = CoordPair((t2.shape[-1]-1)//2, (t2.shape[-2]-1)//2)
angle = 0.0

t2_line = interpline(t2, pivot_point=pivot_point, angle=angle, latlon=True)
t2_line

### 5. 非规则网格到规则网格插值

由于 WRF 模式模拟时的投影设置不一定是规则网格，例如基于兰伯特投影的 WRF 模拟需要进行重采样，以实现非规则网格到规则网格插值。

In [ ]:
# 设置输出的网格属性（可以根据需求自行设置分辨率）
ds_out = xr.Dataset({"lat": ("lat", np.arange(22, 32+0.025, 0.025)),
                     "lon": ("lon", np.arange(117, 129+0.025, 0.025))})
ds_out

In [ ]:
# 用 salem 读取 WRF 模拟数据
ds_in = salem.open_wrf_dataset(wrfout_path + 'wrfout_d01_2019-08-09_06_00_00')

# 使用双线性方法执行重插值
regridder = xe.Regridder(ds_in=ds_in, ds_out=ds_out, method='bilinear')
regridder

In [ ]:
# 得到2m温度变量（T2）的重插值结果
t2_grids = regridder(ds_in.T2.isel(time=0))
t2_grids

In [ ]:
# 绘制2m温度变量（T2）的重插值结果
# 此时经纬度能够直接显示出来
t2_grids.plot(figsize=(10,6))

## 三、站点与格点位置的转换


**WRF模拟的经纬度数据与格点位置数组是分开存储的，两者之间进行相互转换，可以为我们实现从格点中提取站点模拟值的功能。**  

根据站点位置得到格点位置：`wrf.ll_to_xy`   （输入为经纬度、输出为格点位置）  

根据格点位置得到站点位置：`wrf.xy_to_ll`    （输入为格点位置、输出为经纬度）  

**注意：站点与格点位置转换本身并不涉及插值！！所以我们提取到站点模拟值就是原始格点模拟值。**

### 1. 根据站点位置得到格点位置

In [ ]:
# 得到纬度为25°N，经度为120°E所在WRF模拟的格点位置
ll_to_xy(wrf_file, 25, 120)

### 2. 根据格点位置得到站点位置

In [ ]:
# 获取海平面气压最低值所在的格点位置
slp = getvar(wrf_file, 'slp')
loc = np.where(np.array(slp == slp.min()) == True)
loc

In [ ]:
# 根据格点位置得到该点的经纬度信息
xy_to_ll(wrf_file, 221, 203)

这样我们就知道海平面气压最低值的经纬度数据啦，位于26.58969234°N, 122.93884283°E，也就是台风中心位置。

恭喜你完成了WRF后处理训练营第五关的学习材料，虽然本关卡内容涉及较多、技术性较强，但本质都是围绕**格点与格点、格点与站点**之间的转换进行的，为了让数据维度更符合我们业务和研究的需要。

## 闯关题  

### STEP1：根据要求完成题目：  


基于`wrfout_d01_2019-08-09_06_00_00`模拟数据，  

1. 将海平面气压分别重插值到0.01°和0.1°分辨率，比较插值后与原始的海平面最低气压（slp_min）的变化，单位为Pa进行取整后比较  
A. 0.01°分辨率：slp_min**变高**；0.1°分辨率：slp_min**变高**  
B. 0.01°分辨率：slp_min**变低**；0.1°分辨率：slp_min**变低**  
C. 0.01°分辨率：slp_min**不变**；0.1°分辨率：slp_min**变高**  
D. 0.01°分辨率：slp_min**变高**；0.1°分辨率：slp_min**不变**  

2. 热带气旋利奇马中心（海平面气压最低值）所在位置的500hPa相对湿度是多少？（使用int函数取整，单位为%）   
提示：在确定热带气旋中心点时，south_north 对应 Y 表示 lat，west_east 对应 X 表示 lon  

3.  上海（121°E，31°N）所在的WRF格点位置的索引值是多少？（分别为west_east东西向索引值和south_north南北向索引值）  


In [ ]:
# 填入你的答案
answer_1 = ''
answer_2 = '' 
answer_3 = '' # 用数值表示索引，例如'100,100'

## ⚡ 下一关预告：  

下一关我们将介绍如何绘制 WRF 模拟的天气图，让你的 WRF 模拟分析能力更上一层楼~~